In [1]:
import pandas as pd
import numpy as np
from preprocessing import preprocess
import h5py
import urllib.request
import re
from maayanlab_bioinformatics.harmonization import ncbi_genes
from get_symbol import get_symbol
from prediction_pipeline import prediction_pipeline

/opt/apps/rhel7/Python-GPU-3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
f = pd.read_csv('GSE156063_swab_gene_counts.csv.gz')
f.index = f.iloc[:, 0]
f = f.iloc[:, 1:]

In [3]:
norm = preprocess(f)

In [4]:
cor = np.corrcoef(norm)

In [5]:
cor = pd.DataFrame(cor, index=f.index, columns=f.index)

# Saving info

In [24]:
gene_list = (pd.DataFrame(f.index)).values.astype("S").tolist()

In [21]:
h = h5py.File('correlation.h5', 'w')

In [25]:
h.create_dataset("cor", data=cor)
h.create_dataset("genes", data=gene_list)

<HDF5 dataset "genes": shape (15979, 1), type "|S15">

In [28]:
h.close()

# Gene set libraries

In [6]:
libraries = ['GO_Biological_Process_2018', 'GO_Molecular_Function_2018', 'GO_Cellular_Component_2018']

In [7]:
cor = get_symbol(cor)

In [8]:
prediction_pipeline("correlation.h5", libraries, cor)

In [9]:
f = h5py.File("correlation.h5", "r")

In [10]:
list(f.keys())

['cor',
 'cor_GO_Biological_Process_2018',
 'cor_GO_Cellular_Component_2018',
 'cor_GO_Molecular_Function_2018',
 'genes',
 'genes_GO_Biological_Process_2018',
 'genes_GO_Cellular_Component_2018',
 'genes_GO_Molecular_Function_2018',
 'pathways_GO_Biological_Process_2018',
 'pathways_GO_Cellular_Component_2018',
 'pathways_GO_Molecular_Function_2018']